In [ ]:
# from models.detection.ofa_resnet50_fasterrcnn import get_ofa_resnet50_fasterrcnn_model, load_pretrained_fasterrcnn, set_training_params
# model = get_ofa_resnet50_fasterrcnn_model()
# print(model)

from models.detection.ofa_mbv3_w12_fcos import get_ofa_mbv3_w12_fcos_model, load_pretrained_fcos, set_training_params
model = get_ofa_mbv3_w12_fcos_model()
print(model)

In [ ]:
# 只保存部分权重
weight_to_save = ['backbone.dynamic_convs', 'backbone.fpn', 'rpn', 'roi_heads', 'head']

In [ ]:
import torch

# 保存指定的层
def save_selected_layers(model, layers_to_save, save_path):
    state_dict = {}
    for name, param in model.named_parameters():
        # 检查该参数是否属于我们要保存的层
        if any(layer in name for layer in layers_to_save):
            state_dict[name] = param
    torch.save(state_dict, save_path)

# 加载指定的层
def load_selected_layers(model, checkpoint_path):
    state_dict = torch.load(checkpoint_path)
    model_state_dict = model.state_dict()
    
    # 只更新checkpoint中存在的层
    for name, param in state_dict.items():
        if name in model_state_dict:
            model_state_dict[name] = param
            print(f'Loaded {name}')
    
    model.load_state_dict(model_state_dict)


In [ ]:
save_path = 'selected_layers.pth'
save_selected_layers(model, weight_to_save, save_path)
load_selected_layers(model, save_path)

In [ ]:
from thop import profile
import torch
import torch.nn as nn

subnet_config = model.backbone.body.sample_active_subnet()
model.backbone.body.set_active_subnet(**subnet_config)
# model.backbone.body.set_max_net()

# from models.backbone.ofa_supernet import get_ofa_supernet_mbv3_w12
# model = get_ofa_supernet_mbv3_w12()

# subnet_config = model.sample_active_subnet()
# model.set_active_subnet(**subnet_config)

print(subnet_config)
input = torch.randn(1, 3, 128, 128)
flops, params = profile(model, inputs=(input, ))
print(f"Total FLOPs: {flops}")